In [3]:
import requests
import json

def create_session():
    """Создает сессию с правильными заголовками"""
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 HeadHunter Research Tool/1.0",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
    })
    return session

def get_vacancy_details(vacancy_id):
    """Получает детальную информацию о вакансии"""
    session = create_session()
    url = f"https://api.hh.ru/vacancies/{vacancy_id}"
    
    try:
        response = session.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Ошибка при получении вакансии: {e}")
        return None

def print_nested_keys(data, prefix=""):
    """Рекурсивно печатает структуру объекта с вложенными ключами"""
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)) and value:  # Если не пустой словарь или список
                print(f"{prefix}{key}: {type(value).__name__}")
                if isinstance(value, dict):
                    print_nested_keys(value, prefix + "  ")
                elif isinstance(value, list) and len(value) > 0:
                    # Показываем только первый элемент списка, если он словарь
                    if isinstance(value[0], dict):
                        print(f"{prefix}  [0]: (всего элементов: {len(value)})")
                        print_nested_keys(value[0], prefix + "    ")
                    else:
                        print(f"{prefix}  [список из {len(value)} элементов: {value[:3]}...]")
            else:
                print(f"{prefix}{key}: {value}")

def main():
    # Получаем ID вакансии из поиска
    session = create_session()
    search_url = "https://api.hh.ru/vacancies?text=программист&area=1&per_page=1"
    
    try:
        search_response = session.get(search_url)
        search_response.raise_for_status()
        search_data = search_response.json()
        
        if not search_data.get("items"):
            print("Вакансии не найдены.")
            return
            
        vacancy_id = search_data["items"][0]["id"]
        print(f"Получена вакансия с ID: {vacancy_id}")
        
        # Получаем детальную информацию о вакансии
        vacancy = get_vacancy_details(vacancy_id)
        
        if not vacancy:
            print("Не удалось получить данные о вакансии.")
            return
            
        print("\n\n=== СТРУКТУРА ВАКАНСИИ ===\n")
        print_nested_keys(vacancy)
        
        # Проверяем наличие поля specializations
        print("\n\n=== ПРОВЕРКА ПОЛЯ SPECIALIZATIONS ===\n")
        if "specializations" in vacancy:
            if vacancy["specializations"]:
                print(f"Найдены специализации: {json.dumps(vacancy['specializations'], ensure_ascii=False, indent=2)}")
            else:
                print("Поле specializations существует, но пустое.")
                
                # Проверяем дополнительные поля, которые могут содержать информацию о специализации
                print("\nДругие поля, которые могут содержать информацию о специализации:")
                
                if "professional_roles" in vacancy and vacancy["professional_roles"]:
                    print(f"professional_roles: {json.dumps(vacancy['professional_roles'], ensure_ascii=False, indent=2)}")
                
                if "department" in vacancy and vacancy["department"]:
                    print(f"department: {json.dumps(vacancy['department'], ensure_ascii=False, indent=2)}")
                
                if "employer" in vacancy and vacancy["employer"]:
                    print(f"employer: {json.dumps({k: v for k, v in vacancy['employer'].items() if k != 'logo_urls'}, ensure_ascii=False, indent=2)}")
        else:
            print("Поле specializations отсутствует в вакансии.")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

if __name__ == "__main__":
    main()

Получена вакансия с ID: 120721647


=== СТРУКТУРА ВАКАНСИИ ===

id: 120721647
premium: False
billing_type: dict
  id: standard
  name: Стандарт
relations: []
name: Frontend-разработчик
insider_interview: None
response_letter_required: False
area: dict
  id: 1
  name: Москва
  url: https://api.hh.ru/areas/1
salary: None
salary_range: None
type: dict
  id: open
  name: Открытая
address: None
allow_messages: True
experience: dict
  id: between1And3
  name: От 1 года до 3 лет
schedule: dict
  id: remote
  name: Удаленная работа
employment: dict
  id: project
  name: Проектная работа
department: None
show_contacts: False
contacts: None
description: <p><strong>О компании:</strong></p> <p>Staffjet — это компания, которая формирует поток анкет кандидатов на линейные позиции при массовом подборе персонала. Наша экосистема включает CPA-сеть для кадровой лидогенерации, отдел рекламы и кол-центр.</p> <p><strong>О проекте:</strong></p> <p>Работа над развитием SaaS web-платформы для автоматизации уп

In [4]:
import requests
import json

# Получаем ID вакансии из поиска
r1 = requests.get("https://api.hh.ru/vacancies?text=программист&area=1&per_page=1")
search_data = r1.json()
vacancy_id = search_data["items"][0]["id"]
print(f"Получена вакансия с ID: {vacancy_id}")

# Получаем детальную информацию
r2 = requests.get(f"https://api.hh.ru/vacancies/{vacancy_id}")
vacancy = r2.json()

# Проверяем ключи в вакансии
print(f"Ключи вакансии: {list(vacancy.keys())}")

# Проверяем поле specializations
print(f"Есть ли поле specializations? {'specializations' in vacancy}")
specializations = vacancy.get("specializations", [])
print(f"Содержимое поля specializations: {json.dumps(specializations, ensure_ascii=False, indent=2)}")

# Проверяем поле professional_roles
print(f"Есть ли поле professional_roles? {'professional_roles' in vacancy}")
professional_roles = vacancy.get("professional_roles", [])
print(f"Содержимое поля professional_roles: {json.dumps(professional_roles, ensure_ascii=False, indent=2)}")

Получена вакансия с ID: 120721647
Ключи вакансии: ['id', 'premium', 'billing_type', 'relations', 'name', 'insider_interview', 'response_letter_required', 'area', 'salary', 'salary_range', 'type', 'address', 'allow_messages', 'experience', 'schedule', 'employment', 'department', 'show_contacts', 'contacts', 'description', 'branded_description', 'vacancy_constructor_template', 'key_skills', 'accept_handicapped', 'accept_kids', 'archived', 'response_url', 'specializations', 'professional_roles', 'code', 'hidden', 'quick_responses_allowed', 'driver_license_types', 'accept_incomplete_resumes', 'employer', 'published_at', 'created_at', 'initial_created_at', 'negotiations_url', 'suitable_resumes_url', 'apply_alternate_url', 'has_test', 'test', 'alternate_url', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'languages', 'approved', 'employment_form', 'fly_in_fly_out_duration', 'internship', 'night_shifts', 'work_format', 'work_schedule_by_days', 'working_ho

In [5]:
import requests

# Получаем ID работодателя из вакансии
vacancy_id = "120721647"  # ID из вашего примера
vacancy_response = requests.get(f"https://api.hh.ru/vacancies/{vacancy_id}")
vacancy_data = vacancy_response.json()
employer_id = vacancy_data["employer"]["id"]

# Получаем информацию о работодателе
employer_response = requests.get(f"https://api.hh.ru/employers/{employer_id}")
employer_data = employer_response.json()

# Проверяем наличие поля industries
print(f"Ключи работодателя: {list(employer_data.keys())}")
if "industries" in employer_data:
    print(f"Отрасли работодателя: {employer_data['industries']}")
else:
    print("Поле industries отсутствует у работодателя")

Ключи работодателя: ['id', 'trusted', 'accredited_it_employer', 'name', 'type', 'description', 'site_url', 'alternate_url', 'vacancies_url', 'logo_urls', 'relations', 'area', 'industries', 'branded_description', 'branding', 'insider_interviews', 'open_vacancies']
Отрасли работодателя: [{'id': '44.391', 'name': 'Кадровые агентства'}]
